# Introdução
___
Este arquivo tem por objetivo realizar o tratamento de dados do arquivo `cs_bisnode_panel.csv`.

### Objetivos do tratamento de dados:
* Remova as colunas ['COGS', 'finished_prod', 'net_dom_sales', 'net_exp_sales', 'wages', 'D']  pois elas apresentam um percentual considerável de missing data ✅
* Remova de seus dados os registros do ano de 2016 ✅
* Criar uma coluna para variavel resposta (use o conceito de que uma empresa deixou de operar se ela esteve ativa no
ano X, mas não apresentou vendas em X + 2 anos) `# trabalhar nisso`
* Filtre para trabalhar apenas com empresas do ano de 2012 ✅
* usar np.where para ajustar Sales < 0 você já pode substituir por 0 ✅
* * Criar uma nova coluna para a escala logaritima de `Sales`✅
* Essa variável (`Sales`) é bastante assimétrica, concorda? Será que vale criar novas
colunas que representem o valor em log  dessa coluna?✅
* Será que isso também se aplica para as demais? `checar isso durante o loop de retreinamento`
* Crie novas colunas, como idade da empresa (faça isso pela subtração de
founded_year  e year ). Ah, cuide bem dos missing values. np.where pode ajudar
bastante!✅
* Filtre seus dados para ter empresas que possuem receita (revenue) abaixo de 10
milhões de euros e acima de 1000 euros✅
* Busque sempre embasar qualquer decisão de tratamento das variáveis. Faça isso
com o auxílio de estatísticas descritivas e também de gráficos de apoio.✅


# Importação dos dados e tratamento

In [31]:
import pandas as pd
import numpy as np

In [32]:
#pip install missingno

In [33]:
import missingno as msno

In [34]:
df = pd.read_csv("cs_bisnode_panel.csv")
df.head()

,comp_id,begin,end,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,gender,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg
0,1001034.0,2005-01-01,2005-12-31,NaN,692.592590,7266.666504,7574.074219,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
1,1001034.0,2006-01-01,2006-12-31,NaN,603.703674,13122.222656,12211.111328,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
2,1001034.0,2007-01-01,2007-12-31,NaN,425.925934,8196.295898,7800.000000,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
3,1001034.0,2008-01-01,2008-12-31,NaN,300.000000,8485.185547,7781.481445,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
4,1001034.0,2009-01-01,2009-12-31,NaN,207.407410,5137.037109,15300.000000,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,0.083333


In [35]:
#msno.matrix(df)

## Removendo dados do ano de 2016
___

### convertendo as colunas para o formato datetime: 

In [36]:
df["begin"] = pd.to_datetime(df['begin'])
df["end"] = pd.to_datetime(df['end'])

In [37]:
# retinrando os dados em que o ano seja 2016 ou maior
df = df[df["begin"] < "2016"]
df.head()

,comp_id,begin,end,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,gender,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg
0,1001034.0,2005-01-01,2005-12-31,NaN,692.592590,7266.666504,7574.074219,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
1,1001034.0,2006-01-01,2006-12-31,NaN,603.703674,13122.222656,12211.111328,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
2,1001034.0,2007-01-01,2007-12-31,NaN,425.925934,8196.295898,7800.000000,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
3,1001034.0,2008-01-01,2008-12-31,NaN,300.000000,8485.185547,7781.481445,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
4,1001034.0,2009-01-01,2009-12-31,NaN,207.407410,5137.037109,15300.000000,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,0.083333


## Criando a coluna indicadora de operação da empresa 
(se a empresa está operante em x+2 anos ou Não)

In [38]:
# Ordenando o DataFrame original por 'comp_id' e 'begin' para garantir a ordem cronológica
df_sorted = df.sort_values(by=['comp_id', 'begin'])

# Criando a variável resposta 'operates_within_2_years' no DataFrame ordenado
df_sorted['operates_within_2_years'] = df_sorted.groupby('comp_id')['sales'].shift(-2).gt(0).astype(int)

# Adicionando a nova coluna 'operates_within_2_years' de df_sorted ao df original
df['operates_within_2_years'] = df_sorted['operates_within_2_years']

In [39]:
# Verificando as primeiras linhas do df para garantir que a coluna foi adicionada corretamente
df[["comp_id", "sales", "operates_within_2_years"]].head(30)

,comp_id,sales,operates_within_2_years
0,1001034.0,6.275185e+04,1
1,1001034.0,6.462593e+04,1
2,1001034.0,6.510000e+04,1
3,1001034.0,7.808519e+04,1
4,1001034.0,4.538889e+04,0
5,1001034.0,9.929630e+03,0
6,1001034.0,0.000000e+00,0
7,1001034.0,0.000000e+00,0
8,1001034.0,0.000000e+00,0
9,1001034.0,0.000000e+00,0


* `groupby('comp_id')`: Agrupa os dados por empresa usando comp_id para que o cálculo de sales dois anos à frente seja feito separadamente para cada empresa.
* `.shift(-2)`: Pega o valor de sales duas linhas à frente dentro de cada grupo, o que equivale a dois anos à frente, devido à ordenação das datas.
* `apply(lambda x: 1 if x > 0 else 0)`: Define 1 se o valor for positivo e 0 caso contrário.

## Filtro para trabalhar apenas com empresas do ano de 2012
___

In [40]:
df = df[df['begin'] == "2012"]

In [41]:
df.head()

,comp_id,begin,end,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg,operates_within_2_years
7,1001034.0,2012-01-01,2012-12-31,NaN,140.740738,148.148148,21429.628906,0.0,0.0,0.0,...,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,0.083333,0
14,1001541.0,2012-01-01,2012-12-31,NaN,481.481476,9629.629883,1303.703735,0.0,0.0,0.0,...,Domestic,5610.0,56.0,3.0,3,Central,2008-02-24,NaN,NaN,1
23,1002029.0,2012-01-01,2012-12-31,NaN,14929.629883,203885.187500,120444.453125,0.0,0.0,0.0,...,Domestic,2711.0,27.0,2.0,3,East,2006-07-03,NaN,0.458333,1
35,1003200.0,2012-01-01,2012-12-31,NaN,25.925926,22.222221,10996.295898,0.0,0.0,0.0,...,Domestic,5630.0,56.0,3.0,1,Central,2003-10-21,2014-08-09,NaN,0
48,1007261.0,2012-01-01,2012-12-31,NaN,0.000000,255.555557,9207.407227,0.0,0.0,0.0,...,Domestic,5610.0,56.0,3.0,1,Central,2010-08-26,2015-11-19,0.083333,0


In [42]:
import numpy as np

# Verificar se as colunas necessárias estão presentes no DataFrame
required_columns = ['liq_assets', 'curr_liab', 'profit_loss_year', 'sales', 'share_eq']
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"As seguintes colunas estão faltando no DataFrame: {missing_columns}")
else:
    # Criando as colunas derivadas e lidando com NaN ou infinito
    # Razão de Liquidez: Se houver divisão por zero ou NaN, o resultado será substituído por 0
    df.loc[:, 'liquidity_ratio'] = np.where(
        df['curr_liab'] == 0, 0, df['liq_assets'] / df['curr_liab']
    )
    df['liquidity_ratio'].replace([np.inf, -np.inf, np.nan], 0, inplace=True)

    # Margem de Lucro: Lidando com valores faltantes ou divisões por zero
    df.loc[:, 'profit_margin'] = np.where(
        df['sales'] == 0, 0, df['profit_loss_year'] / df['sales']
    )
    df['profit_margin'].replace([np.inf, -np.inf, np.nan], 0, inplace=True)

    # Alavancagem: Substituindo divisões por zero ou NaN por 0
    df.loc[:, 'leverage_ratio'] = np.where(
        df['share_eq'] == 0, 0, df['curr_liab'] / df['share_eq']
    )
    df['leverage_ratio'].replace([np.inf, -np.inf, np.nan], 0, inplace=True)

    # Removendo as colunas originais após a criação das variáveis derivadas
    df = df.drop(columns=['liq_assets', 'curr_liab', 'profit_loss_year', 'share_eq'])

    # Verificando as primeiras linhas para garantir o resultado
    print(df.head())


      comp_id      begin        end  COGS         amort    curr_assets  \
7   1001034.0 2012-01-01 2012-12-31   NaN    140.740738     148.148148   
14  1001541.0 2012-01-01 2012-12-31   NaN    481.481476    9629.629883   
23  1002029.0 2012-01-01 2012-12-31   NaN  14929.629883  203885.187500   
35  1003200.0 2012-01-01 2012-12-31   NaN     25.925926      22.222221   
48  1007261.0 2012-01-01 2012-12-31   NaN      0.000000     255.555557   

    extra_exp  extra_inc  extra_profit_loss  finished_prod  ...  ind  urban_m  \
7         0.0        0.0                0.0            NaN  ...  3.0        1   
14        0.0        0.0                0.0            NaN  ...  3.0        3   
23        0.0        0.0                0.0            NaN  ...  2.0        3   
35        0.0        0.0                0.0            NaN  ...  3.0        1   
48        0.0        0.0                0.0            NaN  ...  3.0        1   

    region_m  founded_date   exit_date  labor_avg  operates_within_2

## retirnado as colunas `'COGS', 'finished_prod', 'net_dom_sales','net_exp_sales', 'wages', 'D'` e checando missing em outras colunas
___

Retirando todas as colunas com alta taxa de missing, de acorda com um limite pré-definido:

In [43]:
#Checando as colunas e seus respectivos valores de missing
percent_missing = df.isnull().mean() * 100
missing_columns = percent_missing[percent_missing > 0].sort_values(ascending=False)
print(missing_columns)

D                    100.000000
finished_prod         94.843477
wages                 94.746257
net_exp_sales         94.660704
net_dom_sales         94.660704
COGS                  94.660704
exit_year             88.189773
exit_date             86.863698
birth_year            24.491542
labor_avg             21.578845
founded_year           8.255882
foreign                8.248104
ceo_count              8.248104
origin                 8.248104
gender                 8.248104
inoffice_days          8.248104
female                 8.248104
personnel_exp          3.461015
amort                  3.461015
material_exp           3.461015
inc_bef_tax            3.223799
extra_profit_loss      3.223799
sales                  3.223799
extra_exp              3.223799
extra_inc              3.223799
ind                    3.134357
region_m               0.279992
fixed_assets           0.038888
curr_assets            0.038888
tang_assets            0.038888
intang_assets          0.038888
subscrib

Retirando as colunas com valor de missing acima do limite exceto a coluna `founded_year` que será necessária adiante

In [44]:
limite = 0.1  # limite da taxa de missing

# Excluindo as colunas acima do limite de missing, exceto 'founded_year'
colunas_para_manter = df.columns[df.isnull().mean() <= limite].tolist()

# Garantindo que a coluna 'founded_year' esteja na lista, mesmo que tenha mais de 10% de missing
if 'founded_year' not in colunas_para_manter:
    colunas_para_manter.append('founded_year')

# Criando o DataFrame filtrado
df = df[colunas_para_manter]

# Verificando o DataFrame resultante
print(df)


             comp_id      begin        end         amort   curr_assets  \
7       1.001034e+06 2012-01-01 2012-12-31    140.740738  1.481481e+02   
14      1.001541e+06 2012-01-01 2012-12-31    481.481476  9.629630e+03   
23      1.002029e+06 2012-01-01 2012-12-31  14929.629883  2.038852e+05   
35      1.003200e+06 2012-01-01 2012-12-31     25.925926  2.222222e+01   
48      1.007261e+06 2012-01-01 2012-12-31      0.000000  2.555556e+02   
...              ...        ...        ...           ...           ...   
287779  4.639820e+11 2012-01-01 2012-12-31  16337.037109  3.140000e+04   
287790  4.640212e+11 2012-01-01 2012-12-31   3566.666748  3.149630e+04   
287798  4.640446e+11 2012-01-01 2012-12-31  76059.257812  4.776548e+06   
287814  4.641050e+11 2012-01-01 2012-12-31   1311.111084  3.261481e+04   
287825  4.641209e+11 2012-01-01 2012-12-31      0.000000  1.518519e+03   

        extra_exp  extra_inc  extra_profit_loss   fixed_assets    inc_bef_tax  \
7        0.000000        0.0  

Preenchendo valores faltantes nas colunas que sobraram de acordo com o tipo da coluna:

In [45]:
# Substituir valores Inf por NaN, para que sejam removidos ou preenchidos corretamente
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Estratégia 1: Para colunas numéricas, preencher com a mediana
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].median())

# Estratégia 2: Para colunas categóricas, preencher com a moda (valor mais frequente)
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
df[categorical_cols] = df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0])

# Verificando o DataFrame resultante
print(df.isnull().sum())  # Deve retornar 0 missing values se o preenchimento foi bem sucedido


comp_id                    0
begin                      0
end                        0
amort                      0
curr_assets                0
extra_exp                  0
extra_inc                  0
extra_profit_loss          0
fixed_assets               0
inc_bef_tax                0
intang_assets              0
inventories                0
material_exp               0
personnel_exp              0
sales                      0
subscribed_cap             0
tang_assets                0
balsheet_flag              0
balsheet_length            0
balsheet_notfullyear       0
year                       0
founded_year               0
ceo_count                  0
foreign                    0
female                     0
inoffice_days              0
gender                     0
origin                     0
nace_main                  0
ind2                       0
ind                        0
urban_m                    0
region_m                   0
founded_date               0
operates_withi

Como a base de dados possuí um número alto de colunas, fizemos um filtro mais rigoroso para missing, uma vez que isso não afetará o numero de variavéis para treinamento do modelo de forma muito significativa, e nos garantirá um modelo melhor por ser treinado em um banco de dados com baixa taxa de missing.

In [46]:
# Checando a taxa de missing nas colunas após filtragem
percent_missing = df.isnull().mean() * 100
missing_columns = percent_missing[percent_missing > 0].sort_values(ascending=False)
print(missing_columns)

Series([], dtype: float64)


In [47]:
# checando as colunas presentas no dataframe para ver quantas variaveis teremos para trabalhar em nosso modelo
df.shape[1]

38

## Trabalhando as incosistencias
___

### Ajustando a coluna sales

### Filtrando os dados para empresas com revenue abaixo de 10 milhões
Como não há uma coluna `revenue`, iremos considerar que `sales` corresponde à receita da empresa

In [48]:
df['sales'].describe()

count    2.571500e+04
mean     4.039782e+05
std      3.216613e+06
min     -1.834445e+04
25%      5.555556e+03
50%      2.923519e+04
75%      1.018407e+05
max      1.058662e+08
Name: sales, dtype: float64

In [49]:
df['sales'] = np.where(df['sales'] < 0, 0, df['sales']) # retirando valores negativos
df = df[(df['sales'] > 1000) & (df['sales'] < 10_000_000)] # filtrando os dados

In [50]:
df['sales'].describe()

count    2.106900e+04
mean     2.299185e+05
std      7.592216e+05
min      1.003704e+03
25%      1.651852e+04
50%      4.148518e+04
75%      1.311296e+05
max      9.919548e+06
Name: sales, dtype: float64

Checando assimetria nas outras colunas:

### Checando a assimetria da coluna `Sales`

In [51]:
df["sales"].skew()

7.356971830831772

Como a skewness esta acima de zero (bem acima) temos que a coluna `sales` possuí alta assímetria positiva

Criando uma nova coluna para a escala logaritima de `sales`

In [52]:
df["sales"] = np.log1p(df['sales'])

In [53]:
df["sales"].skew()

0.3230199174478501

Agora temos que a escala logaritima da coluna `sales_log` possue sua assimetria bem mais próxima de zero em relação à `sales`, indicando uma alta redução na assimetria

Checando a assimetria de outras colunas do dataset

In [54]:
# Verificando a assimetria (skewness) de todas as colunas numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns
skewed_cols = df[numeric_cols].skew().sort_values(ascending=False)

# Exibir as colunas com alta assimetria positiva (por exemplo, acima de 1)
print("Colunas com alta assimetria:")
print(skewed_cols[skewed_cols > 1])

# Criando uma lista para armazenar colunas que devem ser removidas
colunas_para_remover = []

# Aplicando a transformação logarítmica apenas às colunas com alta assimetria e sem valores negativos
for col in skewed_cols[skewed_cols > 1].index:
    if (df[col] >= 0).all():  # Verificar se todos os valores são >= 0
        df[col] = np.log1p(df[col])  # Usando log1p para evitar problemas com valores zero ou negativos
    else:
        print(f"Coluna '{col}' contém valores negativos e será removida.")
        colunas_para_remover.append(col)

# Verificando a nova assimetria após a transformação
new_skewed_cols = df[numeric_cols].skew().sort_values(ascending=False)
print("Assimetria após transformação logarítmica:")
print(new_skewed_cols)

# Remover colunas que possuem alta assimetria após a transformação
for col in new_skewed_cols[new_skewed_cols > 1].index:
    print(f"Coluna '{col}' ainda possui alta assimetria e será removida.")
    colunas_para_remover.append(col)

# Remover as colunas do DataFrame
df = df.drop(columns=colunas_para_remover)

# Exibir as colunas restantes
print("Colunas restantes no DataFrame:")
print(df.columns)

Colunas com alta assimetria:
extra_exp               144.239349
extra_inc               131.848404
profit_margin           119.142705
liquidity_ratio         113.729944
extra_profit_loss        70.527671
tang_assets              49.958750
intang_assets            47.803916
fixed_assets             44.407212
inventories              40.124223
subscribed_cap           35.679997
amort                    27.631670
leverage_ratio           25.717709
curr_assets              15.651840
balsheet_notfullyear     12.613409
personnel_exp            12.395652
material_exp              8.256721
foreign                   2.938839
ceo_count                 2.817340
female                    1.220244
inoffice_days             1.154225
dtype: float64
Coluna 'extra_exp' contém valores negativos e será removida.
Coluna 'extra_inc' contém valores negativos e será removida.
Coluna 'profit_margin' contém valores negativos e será removida.
Coluna 'liquidity_ratio' contém valores negativos e será removida.
Co

### Criando a coluna `Idade_da_empresa`

In [55]:
df['Idade_da_empresa'] = np.where(
    (df['year'].isna()) | (df['founded_year'].isna()),  # Condição: se 'year' ou 'founded_year' forem NaN
    np.nan,  # Se a condição for verdadeira (há missing values), preenche com NaN
    df['year'] - df['founded_year']  # Caso contrário, realiza a subtração normalmente
)
df['Idade_da_empresa'].head()

14     4.0
23     6.0
35     9.0
56    20.0
68    11.0
Name: Idade_da_empresa, dtype: float64

### Codificando as colunas não numéricas para serem utilizadas nos modelos: 

In [56]:
# 2. Codificar 'region_m' (aqui estou usando Label Encoding)
# Se houver poucas categorias, podemos usar Label Encoding
df['region_m_encoded'] = df['region_m'].astype('category').cat.codes

# Alternativamente, para One-Hot Encoding:
# df = pd.get_dummies(df, columns=['region_m'], drop_first=True)

# 3. Codificar 'founded_date' extraindo o ano
df['founded_year'] = pd.to_datetime(df['founded_date']).dt.year

# Agora, removemos as colunas originais que foram codificadas
df = df.drop(columns=['begin', 'end', 'region_m', 'founded_date'])

### Checando colunas com números nulos:

In [57]:
#colunas = ["balsheet_length", "year","comp_id","ind","ind2","urban_m"]
dadosR = df.drop(columns="comp_id")
dadosR.head()

,fixed_assets,inc_bef_tax,sales,tang_assets,balsheet_flag,balsheet_length,year,founded_year,inoffice_days,gender,origin,nace_main,ind2,ind,urban_m,operates_within_2_years,Idade_da_empresa,region_m_encoded
14,12.157763,-7696.296387,7.093989,12.157763,0,365,2012,2008,8.078068,male,Domestic,5610.0,56.0,3.0,3,1,4.0,0
23,10.063063,11818.518555,13.943478,10.063063,0,365,2012,2006,8.039802,male,Domestic,2711.0,27.0,2.0,3,1,6.0,1
35,0.000000,-2337.037109,7.932429,0.000000,0,365,2012,2003,7.470224,female,Domestic,5630.0,56.0,3.0,1,0,9.0,0
56,13.746777,96751.851562,12.980034,13.745129,0,365,2012,1992,8.732143,male,Domestic,5510.0,55.0,3.0,2,1,20.0,2
68,11.680393,-1429.629639,11.773216,11.680393,0,365,2012,2001,8.187299,male,Domestic,5510.0,55.0,3.0,2,1,11.0,0


## Criando o dataset para treinar o modelo

In [58]:
# Verificando o total de valores NaN e imprimindo apenas colunas com NaN
print("Colunas com valores NaN no dataset:")
nan_cols = dadosR.isnull().sum()
print(nan_cols[nan_cols > 0])

# Verificando se há valores Inf no dataset e imprimindo apenas colunas com Inf
print("Colunas com valores Inf no dataset:")
inf_cols = (dadosR == np.inf).sum() + (dadosR == -np.inf).sum()
print(inf_cols[inf_cols > 0])


Colunas com valores NaN no dataset:
Series([], dtype: int64)
Colunas com valores Inf no dataset:
Series([], dtype: int64)


In [59]:
# retirando colunas sem significado aparente
dadosR = dadosR.drop(columns = ["nace_main","ind","ind2","urban_m"])

In [60]:
dadosR.to_csv("dados_para_o_R.csv", index = True)
print(f"O dataset final tem {dadosR.shape[0]} linhas e {dadosR.shape[1]} colunas")

O dataset final tem 21069 linhas e 14 colunas
